## EOG Programmatic Access
* ntlights metadata:  https://eogdata.mines.edu/products/vnl/
* directory url:  https://eogdata.mines.edu/nighttime_light/annual/v21/
* sample link (size 9.7GB):  https://eogdata.mines.edu/nighttime_light/annual/v21/2014/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif.gz
* sample mask link (size 267 MB): https://eogdata.mines.edu/nighttime_light/annual/v21/2014/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average_masked.dat.tif.gz

In [1]:
import requests
import json
import os

In [2]:
import getpass

In [4]:
username = input('Username?')

Username? butch@thinkingmachin.es


In [5]:
password = getpass.getpass('Password?') 

Password? ········


In [7]:
# Retrieve access token
params = {    
    'client_id': 'eogdata_oidc',
    'client_secret': '2677ad81-521b-4869-8480-6d05b9e57d48',
    'username': username,
    'password': password,
    'grant_type': 'password'
}

In [8]:
token_url = 'https://eogauth.mines.edu/auth/realms/master/protocol/openid-connect/token'

In [9]:
response = requests.post(token_url, data = params)

In [10]:
access_token_dict = json.loads(response.text)

In [12]:
access_token = access_token_dict.get('access_token')

In [14]:
# Submit request with token bearer
## Change data_url variable to the file you want to download
data_url = 'https://eogdata.mines.edu/eog/EOG_sensitive_contents'

In [15]:
auth = 'Bearer ' + access_token
headers = {'Authorization' : auth}

In [16]:
response = requests.get(data_url, headers = headers)
# Write response to output file
## You can either define the output file name directly
# output_file = 'EOG_sensitive_contents.txt'
## Or get the filename from the data_url variable

In [19]:
from pprint import pprint

In [21]:
contents = response.content

In [22]:
len(contents)

1250

In [24]:
print(str(contents,'utf-8'))

 ====  Welcome to EOG Data Distribution!!  =====

WWWWWWWWWWWWWWWWWWNNMBBBBBBBMNNWWWWWWWWWWWWWWWWWW
WWWWWWWWWWWWWNM&&ggggggggggggggg0&MNWWWWWWWWWWWWW
WWWWWWWWWWNB0ggggggggggg$a$ggggggggg0BNWWWWWWWWWW
WWWWWWWWM&gggggggggggggPr^=jamggSZ98ggg&MWWWWWWWW
WWWWWWB0ggggggggggggggQw7\^^^>o}F#Zvqgggg0BWWWWWW
WWWWN&gggggggggggggggggggdo)*r}#gg2Lbgggggg&NWWWW
WWWN0ggggggggggggggggggggggQZvFRQc*JoHgggggg0NWWW
WWN0ggggggggggggggggggggkc*>rr>xjL|^^\72Dgggg0NWW
WN0gggggggggggggggggggggUur^- ,Jgmac=^^>aggggg0NW
WBggg8RHqSUUSpH$Qggggggg5iwic~;JggggR%udgggggggBW
W0gSkkEUSSppSU6kPPaPU$gg$j?ag$P$ggggggggggggggg&W
N0Q$QDggggggggduywUdSPZaSQggggggggggggggggggggg0N
N0$)rrsggggggUyrrr>rviwHqZ2SQgggggggggggggggggg0N
W0gk?>}DgggDEs)>r>>>>>>iQgmkIUggggggggggggggggg&W
WBgRc>ogg$yv>Lk5y>>>rruRdy$gma%RgggggggggggggggBW
WN0mj>*FZ7>>>>**rr*r>omgF)wSggHFSggggggggggggg&WW
WWN0$J>>>>>>>>>>>xS2rrvi}VRggggQ2qggggggggggg0NWW
WWWN0$ir>>>>>>>>>}u2yU8QggggggggQodggggggggg&NWWW
WWWWN&Q2v>>>>>>>>>rFQQQbqgggq5HggUFQggggggg&WWWWW


In [12]:
# Retrieve access token
def get_eog_access_token(username, password):
    params = {    
        'client_id': 'eogdata_oidc',
        'client_secret': '2677ad81-521b-4869-8480-6d05b9e57d48',
        'username': username,
        'password': password,
        'grant_type': 'password'
    }
    token_url = 'https://eogauth.mines.edu/auth/realms/master/protocol/openid-connect/token'
    response = requests.post(token_url, data = params)
    access_token_dict = json.loads(response.text)
    access_token = access_token_dict.get('access_token')
    return access_token

In [26]:
def download_eog_file(url, access_token, chunk_size=8192, local_filename = None):
    # see https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
    auth = 'Bearer ' + access_token
    headers = {'Authorization' : auth}
    local_filename = url.split('/')[-1] if local_filename is None else local_filename
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True, headers=headers) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=chunk_size): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename 

In [27]:
download_file(data_url,access_token)

'EOG_sensitive_contents'

In [28]:
masked_tif = 'https://eogdata.mines.edu/nighttime_light/annual/v21/2014/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average_masked.dat.tif.gz'

In [31]:
%%time
download_file(masked_tif,access_token, chunk_size=8192*1024*10)

CPU times: user 4.85 s, sys: 3.87 s, total: 8.73 s
Wall time: 9min 43s


'VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average_masked.dat.tif.gz'

In [ ]:
# output_file = os.path.basename(data_url)

# with open(output_file,'wb') as f:
#     f.write(response.content)

In [33]:
(9.7 * 1024)/(267)  * 9.75 # appx 10min for 267MB equals how many minutes for 9.7GB (chunksize of 80MB)

362.714606741573

In [34]:
362.71 / 60 # hours

6.045166666666666